
Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# Optimizer Step: Integrate Localization and Intervention

In [ ]:
#@title Import libraries
import transformer_lens
import torch, gc, itertools, functools, tqdm
import pandas as pd
import numpy as np
import torch.nn.functional as F

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

import sys
sys.path.append('/home/jupyter/')
from paraMem.utils import modelHandlers, dataLoaders, gradient, evaluation, localizing, intervening

## Model and Data

In [ ]:
model_type = "gpt-neo-125M"
model = modelHandlers.load_model(model_type=model_type, DEVICE="cuda", lr=0.0, weight_decay=0.01)

## Data

In [ ]:
## mem and non-mem set
mem_nonmem_sets  = dataLoaders.load_pile_splits(f"{model_type}/preds", file_names=["50_50_preds.pt", "0_10_preds.pt"], as_torch=True)
mem_set, non_mem_set = mem_nonmem_sets[0], mem_nonmem_sets[1]
change_keep_dl, _ = dataLoaders.train_test_batching(mem_set, non_mem_set, mem_batch=1, non_mem_batch=10, test_frac=0.0, add_bos=None)
_, k_toks_NI = next(iter(change_keep_dl))
k_toks_NI = k_toks_NI.squeeze(0)

## load perturbed mem set and original mem set
mem_perturbed_sets  = dataLoaders.load_pile_splits(f"{model_type}/perturbed", file_names=["mem_toks.pt", "perturbed_mem_toks.pt"], as_torch=True)
mem_set, perturbed_mem_set = mem_perturbed_sets[0], mem_perturbed_sets[1]
mem_perturb_dl, _ = dataLoaders.train_test_batching(mem_set, perturbed_mem_set, mem_batch=1, non_mem_batch=1, test_frac=0.0, add_bos=None)
c_toks_NI, c_perturb_toks_NI = next(iter(mem_perturb_dl))
c_toks_NI, c_perturb_toks_NI, = c_toks_NI.squeeze(0), c_perturb_toks_NI.squeeze(0)

In [ ]:
c_em_N_orig, k_em_N_orig = evaluation.model_eval(model, c_toks_NI, k_toks_NI, I_range=[50,100])

In [ ]:
n_batches = 20
all_c_em, all_k_em = [],[]
topK_range = [None, 0.001, -0.001]

for topK in topK_range:
    topK_c_em, topK_k_em = [],[]
    change_keep_iter, mem_perturb_iter = iter(change_keep_dl), iter(mem_perturb_dl)
    for n in tqdm.tqdm(range(0, n_batches)):
        _, k_toks_NI = next(change_keep_iter)
        k_toks_NI = k_toks_NI.squeeze(0)

        c_toks_NI, c_perturb_toks_NI = next(mem_perturb_iter)
        c_toks_NI, c_perturb_toks_NI, = c_toks_NI.squeeze(0), c_perturb_toks_NI.squeeze(0)

        model = modelHandlers.load_model(model, lr=1e-05, weight_decay=1.0)
        metric_fn = functools.partial(gradient.contrast_metric, I_range=[49,99], use_perturb=True, c_set_norm=0.1)
        fwd_bwd_fn = functools.partial(gradient.run_contrastive_fwd_bwd, optim_steps=5, topK=topK, grad_norm=None, c_types=["W_K","W_Q","W_V","W_O","W_in","W_out"]) 
        fwd_cache, bwd_cache, topk_idcs = fwd_bwd_fn(model, metric_fn, c_toks_NI, c_perturb_toks_NI, k_toks_NI)

        c_em, k_em = evaluation.model_eval(model, c_toks_NI, k_toks_NI, I_range=[49,99])
        topK_c_em.append(c_em)
        topK_k_em.append(k_em)
    
    all_c_em.append(torch.stack(topK_c_em).flatten())
    all_k_em.append(torch.stack(topK_k_em).flatten())
    
all_c_em = torch.stack(all_c_em)
all_k_em = torch.stack(all_k_em)

In [ ]:
fig, axs = plt.subplots(1, len(all_c_em), figsize=(14, 1.5), gridspec_kw={'hspace': 0.0})
fontsize = 12

topK_range = ["all of the weights", "weights within max 0.1% gradient", "random 0.1% of the weights"]
for i, ax in enumerate(axs):
    
    #c_counts, c_bins = np.histogram(all_c_em[i], bins=50)
    #_ = ax.hist(c_bins[:-1], c_bins, weights=(c_counts / c_counts.sum()), color="red", density=True)

    #k_counts, k_bins = np.histogram(all_k_em[i], bins=50)
    #_ = ax.hist(k_bins[:-1], k_bins, weights=(k_counts / k_counts.sum()), color="blue", density=True)
    #ax2 = ax.twinx()
    
    sns.violinplot(data = [all_c_em[i].numpy(), all_k_em[i].numpy()],palette=['r','b'], jitter=True, ax=ax) #inner="points"

    #sns.kdeplot(all_c_em[i], color="red", ax=ax2, label="memorized")
    #sns.kdeplot(all_k_em[i], color="blue", ax=ax2, label="non-memorized")
    
    #ax2.get_yaxis().set_ticks([])
    #ax2.set_yticklabels([])
    
    if i==0:
        ax.set_ylabel('exact match', fontsize=fontsize)
    #ax2.set_ylabel('', fontsize=fontsize)
    #ax.set_xlabel('exact match (EM) after optimizing', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    ax.set_ylim(-2,52)
    
    topK = topK_range[i]
    ax.set_title(f"Optimizing {topK}", fontsize=fontsize-1, loc="left", x=-0.1)
    #red_patch = mpatches.Patch(color='red', label='memorized')
    #blue_patch = mpatches.Patch(color='blue', label='non-memorized')
    #plt.legend(handles=[red_patch, blue_patch], frameon=False, ncol=1, prop={'size': fontsize-2})
    ax.set_xticks([])
    ax.yaxis.set_major_locator(plt.MaxNLocator(5))
    
    ax.text(0.07, -0.35, f'memorized set\nmean EM: {all_c_em[i].mean(-1):.2f}', color="red", fontsize=fontsize-1, horizontalalignment='left',verticalalignment='bottom', transform=ax.transAxes)
    ax.text(0.54, -0.35, f'non-memorized set\nmean EM: {all_k_em[i].mean(-1):.2f}', color="blue", fontsize=fontsize-1, horizontalalignment='left',verticalalignment='bottom', transform=ax.transAxes)

fig.savefig(f"{dataLoaders.ROOT}/results/unlearning_perturb.pdf", dpi=200, bbox_inches="tight")